In [1]:
import pandas as pd
import numpy as np

## the input data is the spatial joined dataframe from DataFrameProcess.ipynb

In [2]:
df=pd.read_pickle('firehoseTAZ')

In [3]:
df.head()

,time,cate,TAZ
0,2017-01-01 00:02:56,Food,95
1,2017-01-01 00:07:38,Professional & Other Places,95
2,2017-01-01 00:19:20,Food,95
3,2017-01-01 00:19:25,Food,95
4,2017-01-01 00:21:18,Nightlife Spot,95


In [4]:
n=318 # n is the taz number
docs=[[]]*n

In [5]:
for i in range(len(df)):
    taz=df.iloc[i][2]
    cate=df.iloc[i][1]
    docs[taz].append(cate)

In [6]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(docs)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
docs_matrix = [dictionary.doc2bow(doc) for doc in docs]

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(docs_matrix, num_topics=5, id2word = dictionary, passes=1000)

In [8]:
print(ldamodel.print_topics(num_topics=5, num_words=9)) # num_topics are guided by the New York City zoning and land use data [1]

[(0, '0.201*"Food" + 0.180*"Shop & Service" + 0.160*"Outdoors & Recreation" + 0.156*"Professional & Other Places" + 0.134*"Travel & Transport" + 0.104*"Nightlife Spot" + 0.038*"Arts & Entertainment" + 0.018*"College & University" + 0.010*"Event"'), (1, '0.114*"Food" + 0.113*"Shop & Service" + 0.113*"Travel & Transport" + 0.112*"Outdoors & Recreation" + 0.112*"Professional & Other Places" + 0.109*"Nightlife Spot" + 0.109*"Arts & Entertainment" + 0.109*"College & University" + 0.109*"Event"'), (2, '0.222*"Food" + 0.216*"Outdoors & Recreation" + 0.160*"Travel & Transport" + 0.122*"Shop & Service" + 0.121*"Professional & Other Places" + 0.084*"Nightlife Spot" + 0.044*"Arts & Entertainment" + 0.019*"College & University" + 0.014*"Event"'), (3, '0.218*"Outdoors & Recreation" + 0.209*"Travel & Transport" + 0.195*"Food" + 0.132*"Shop & Service" + 0.093*"Professional & Other Places" + 0.080*"Nightlife Spot" + 0.046*"Arts & Entertainment" + 0.017*"College & University" + 0.011*"Event"'), (4, '0.

In [9]:
tazs=[]

In [10]:
for i in range(n):
    topp=[x[1] for x in ldamodel.get_document_topics(docs_matrix[i])]
    tazs.append(topp.index(max(topp)))

### now we have assigned topics to every taz, the result will be fed into GIS interface for visualization